In [9]:
import pandas as pd
import numpy as np
import csv
import os

In [10]:
abs_path = 'ecosys/ecosys_initial_configurations/'

locales = ['cool_temp_maize_soybean', 'warm_temp_maize_soybean_irrigated', 'warm_temp_maize-soybean_dryland']

local = locales[2]

cfg_files_pth = abs_path + local

out_files = ['soil_c','surf_water','soil_temp','flux_soc','soil_water','n_flux','p_flux','temp']

years = 8
growth_strt = 2015

In [11]:
def handle_soil(df: pd.DataFrame, soil_path: str)-> pd.DataFrame:
    soil_prfl_vars = {
    "initial bulk density (Mg m-3,0=water) ": 2,
    "sand contents (kg Mg-1)": 7,
    "silt contents (kg Mg-1)": 8,
    "rock fraction" : 10,
    "ph" : 11,
    "SOC (kg Mg-1)": 14 
    }
    with open(soil_path, 'r') as slfl:
        lines = slfl.readlines()
        for col_n, ln in soil_prfl_vars.items():
            line_fl = lines[ln].strip().split(',')
            line_fl = [x for x in line_fl if x]
            mean = np.mean([float(i) for i in line_fl])
            df[col_n] = mean
            
        
    return df
            
            

def agrg_temp_plant_soil(temp_fl_path: str, planting_path: str, plt_fl_idx: str):
    planting =  ['planting date', 'initial planting density (m-2)', 'seeding depth (m)']
    harvesting = ['harvesting date']
    with open(planting_path, 'r') as plntfl:
        df = pd.read_csv(temp_fl_path)
        line = plntfl.readline()
        planting_data = line.strip().split(',')
        date = planting_data[0]
        planting_data =planting_data[1:]
        
        if len(planting_data) != 2:
            raise Exception("Planting data mismatch")
        
        df[planting[0] + plt_fl_idx ] = date
        df[planting[1] + plt_fl_idx] = planting_data[0]
        df[planting[2] + plt_fl_idx] = planting_data[1]
    
    
    df = handle_soil(df, '../' + cfg_files_pth + '/soil')
    df.to_csv(temp_fl_path, index=False)

### Adding all planting dates to each temperature file. There have been 7 planting dates across each year from 2001 to 2007
#### This exposes temporal variabilities between planting dates and fluxations in the target variables that the models can pick up on 

In [12]:
for i in range(0, years):
    for j in range(1, 7):
        agrg_temp_plant_soil('../datasets/' + local + '/csv_outs/' + str(growth_strt + i) + 'temp.csv',  '../'+ cfg_files_pth + '/planting' + str(j), str(j)  )